In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow
import seaborn as sns
import pandas as pd

from tensorflow import keras

from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import rdMolDescriptors

from sklearn import datasets, metrics
from sklearn.metrics import auc, roc_auc_score, roc_curve, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import StandardScaler


from scipy import interp
from tensorflow.keras.layers import Embedding, Dense 
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint



from tensorflow.keras import backend as K 
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import initializers

2025-06-26 08:29:47.976703: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-26 08:29:48.746259: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-06-26 08:29:48.746331: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2025-06-26 08:29:48.746336: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import os
import torch
import joblib
from sklearn.metrics import roc_auc_score, average_precision_score

# --- Paths ---
SAVE_DIR = './oob_grid_search_rf'
TEST_DATA_PATH = os.path.join(SAVE_DIR, 'RF_test_holdout.pt')
MODEL_PATH = os.path.join(SAVE_DIR, 'random_forest_oob_model.pkl')

# 1) Load hold-out test set
data = torch.load(TEST_DATA_PATH)
X_test = data['features'].numpy()  # torch tensor to numpy array
y_test = data['labels'].numpy()

# 2) Load trained model
best_rf = joblib.load(MODEL_PATH)

# 3) Predict probabilities on test set
y_proba = best_rf.predict_proba(X_test)[:, 1]

# 4) Compute evaluation metrics
test_roc_auc = roc_auc_score(y_test, y_proba)
test_aupr = average_precision_score(y_test, y_proba)

# 5) Print results
print(f"Test ROC AUC: {test_roc_auc:.4f}")
print(f"Test AUPR: {test_aupr:.4f}")

# 6) Optionally save metrics for later use
metrics = {'roc_auc': test_roc_auc, 'avg_precision': test_aupr}
joblib.dump(metrics, os.path.join(SAVE_DIR, 'RF_test_metrics.pkl'))

Test ROC AUC: 0.7574
Test AUPR: 0.7429


['./oob_grid_search_rf/RF_test_metrics.pkl']